In [1]:
%%capture
!pip install huggingsound
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate optimum
!pip install -q ipython-autotime
!pip install moviepy

In [2]:
import torch
import librosa

import numpy as np

from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from huggingsound import SpeechRecognitionModel
from transformers import (
    Speech2TextProcessor,
    Speech2TextForConditionalGeneration,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    AutoModelForSpeechSeq2Seq, 
    AutoProcessor, 
    pipeline
)
from datasets import load_dataset

In [3]:
%load_ext autotime
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

cpu
time: 939 µs (started: 2023-11-18 10:53:14 +00:00)


In [4]:
song = AudioSegment.from_mp3("/kaggle/input/audio-mp3/-  1.mp3")
song.export("lect.wav", format="wav")

<_io.BufferedRandom name='lect.wav'>

time: 1.41 s (started: 2023-11-18 10:53:14 +00:00)


# jonatasgrosman/wav2vec2-large-xlsr-53-russian

In [6]:
LANG_ID = "ru"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

audio = librosa.load('/kaggle/working/lect.wav', sr=16_000)[0]
inputs = processor(audio.reshape(1,-1), sampling_rate=16_000, return_tensors="pt", padding=True)
with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)
print(predicted_sentences)

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

['здравствуйте уважаемые слушатели скажите а что вы знаете авеч наверное то что это болезнь которые боляет проститутки наркоманы геи и преступники тот ктовил себя плохо и неполучат подарок на новый год от детушки морода а еще то что люди с этим страшным диагнозом умирают через пару лет после заражения этим вирусом конечно все что я сейчас сказал является сарказмом и шуткой однако как известно в любой шутке есть и доля правды поэтому сегодня вам предлагается прослушать познавательны аудивлекцию о том что из себя представлет вич и спит как не заразиться данным заболеванием как его диагностируют и как леча бат ждут ответы на многие вопросы которые возможногоздавной интересовали поговорим с вами также и омифах и предрассудках связанных с вич инфекции и начать пожалуй стоят стерминологии что такое вич что такое спит и в чем разница между ними вич расшифровется как вирус имунодефицита человека и является возбудителем вич инфекции медленно прогрессирующего заболевания при попадании в организм

# openai/whisper-large-v2  
whisper-large-v3 не нужна, так как она умеет еще переводить

In [7]:
LANG_ID = "ru"
MODEL_ID = "openai/whisper-large-v2"

processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)
pipe.model = pipe.model.to_bettertransformer()

audio = librosa.load('/kaggle/working/lect.wav', sr=16_000)[0]

result = pipe(audio, generate_kwargs={"language": "russian"})
result

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.26k [00:00<?, ?B/s]

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


{'text': ' Здравствуйте, уважаемые слушатели! Скажите, а что вы знаете о ВИЧ? Наверное, то, что это болезнь, которой болеют проститутки, наркоманы, геи и преступники. Тот, кто вёл себя плохо и не получит подарок на Новый год от Дедушки Мороза. А ещё то, что люди с этим страшным диагнозом умирают через пару лет после заражения этим вирусом. Конечно, все, что я сейчас сказал, является сарказмом и шуткой. Однако, как известно, в любой шутке есть и доля правды. Поэтому сегодня вам предлагается прослушать познавательную аудиолекцию о том, что из себя представляют ВИЧ и СПИД, как не заразиться данным заболеванием, как его диагностируют и как лечат. Вас ждут ответы на многие вопросы, которые возможно вас давно интересовали. Поговорим с вами также и о мифах и предрассудках связанных с ВИЧ инфекцией. И начать пожалуй стоит с терминологии. Что такое ВИЧ? Что такое СПИД? И в чем разница между ними? ВИЧ расшифровывается как вирус иммунодефицита человека и является возбудителем ВИЧ-инфекции, медлен

time: 7min 7s (started: 2023-11-18 11:29:47 +00:00)
